In [596]:
import numpy as np
import pandas as pd

In [597]:
import pulp

In [598]:
def create_integer_vars(_crops_year, df):

    return dict(zip(_crops_year.keys(),
            [pulp.LpVariable(i, 
                                lowBound = df.loc[_crops_year[i], 'int_lb'] if not np.isnan(df.loc[_crops_year[i], 'int_lb']) else None, 
                                upBound  = df.loc[_crops_year[i], 'int_ub'] if not np.isnan(df.loc[_crops_year[i], 'int_ub']) else None,
                                cat='Integer') 
                    for i in _crops_year.keys()]))

In [599]:
def create_binary_vars(_crops_year, df):

    return dict(zip(_crops_year.keys(),
            [pulp.LpVariable(i, 
                                lowBound = df.loc[_crops_year[i], 'bin_lb'] if not np.isnan(df.loc[_crops_year[i], 'bin_lb']) else None, 
                                upBound  = df.loc[_crops_year[i], 'bin_ub'] if not np.isnan(df.loc[_crops_year[i], 'bin_ub']) else None,
                                cat='Binary') 
                    for i in _crops_year.keys()]))

In [600]:
def of_net_income(_cost, _vars, _crops_year):
    return pulp.lpSum([getattr(_cost, _crops_year[i]) * _vars[i] for i in _crops_year])

In [601]:
def of_ua_total(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

In [602]:
def of_crop_diversity(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

In [ ]:
def slice_dict(_crops_year, start, end):
    crops = dict(list(_crops_year.items())[start:end])
    return crops

In [603]:
def add_constraints(_model, _var, _const, _crops_year, n_crops, n_years, const_id = 0):

    for year in range(n_years):

        crops = slice_dict(_crops_year, year * n_crops, (year + 1) * n_crops)
    
        for row in _const.itertuples():

            constraint = pulp.lpSum([getattr(row, _crops_year[i]) * _var[i] for i in crops])
            rhs = getattr(row, 'RHS')

            match getattr(row, 'type'):
                case '<=':
                    constraint = constraint <= rhs
                case '<':
                    constraint = constraint < rhs
                case '>=':
                    constraint = constraint >= rhs
                case '>':
                    constraint = constraint > rhs
                case '==':
                    constraint = constraint == rhs
            
            _model += (constraint, f"C_{const_id}_{getattr(row, 'name')}", )

            const_id += 1
    
    return const_id



In [604]:
def add_binary_correlation(_model, _var_bin, _var_int, min_ua_crop, max_ua_available = (10**10),const_id = 0):
    for var in _var_bin.keys():
        constraint = min_ua_crop * _var_bin[var] - _var_int[var.replace('bin', 'int')]
        _model += (constraint <= 0, f"C_{const_id}_Min_UA_Per_Crop__Relation_Bin_Int", )
        const_id += 1

        constraint = _var_int[var.replace('bin', 'int')] - max_ua_available * _var_bin[var] <= 0
        _model += (constraint <= 0, f"C_{const_id}_Relation_Bin_Int", )
        const_id += 1

    return const_id

In [ ]:
def add_constraints(_model, _var, _const, _crops_year, n_crops, n_years, const_id = 0):


In [619]:
a = 'y{}_int_{}'
a.format(1, 'teste')

'y1_int_teste'

In [ ]:
for year in range(1, n_years):
    

In [618]:
crops_year_integer

{'y1_int_Wheat': 'Wheat',
 'y1_int_Beetroot': 'Beetroot',
 'y1_int_Cloves': 'Cloves',
 'y1_int_Broad_Bean': 'Broad_Bean',
 'y1_int_Green_Onion': 'Green_Onion',
 'y1_int_Autumn_Potatoes': 'Autumn_Potatoes',
 'y1_int_Sunflower': 'Sunflower',
 'y1_int_Maize': 'Maize',
 'y1_int_Tomatoes': 'Tomatoes',
 'y1_int_Cucumbers': 'Cucumbers',
 'y1_int_Okra': 'Okra',
 'y1_int_Watermelon': 'Watermelon',
 'y2_int_Wheat': 'Wheat',
 'y2_int_Beetroot': 'Beetroot',
 'y2_int_Cloves': 'Cloves',
 'y2_int_Broad_Bean': 'Broad_Bean',
 'y2_int_Green_Onion': 'Green_Onion',
 'y2_int_Autumn_Potatoes': 'Autumn_Potatoes',
 'y2_int_Sunflower': 'Sunflower',
 'y2_int_Maize': 'Maize',
 'y2_int_Tomatoes': 'Tomatoes',
 'y2_int_Cucumbers': 'Cucumbers',
 'y2_int_Okra': 'Okra',
 'y2_int_Watermelon': 'Watermelon'}

In [ ]:
for 

In [ ]:
def add_crop_rotation_constraints(_model, _var, _const, n_years, n_crops, string_var, const_id = 0):


In [605]:
variables = pd.read_csv('dados/al_rasheed/variables.csv', sep=';')
income = pd.read_csv('dados/al_rasheed/net_income.csv', sep=';')
constraints = pd.read_csv('dados/al_rasheed/constraints.csv', sep=';')

In [606]:
YEARS = 2

In [607]:
crops = variables.variable.to_list()

In [608]:
crops_year_integer = dict(zip([f"y{i}_int_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

crops_year_binary = dict(zip([f"y{i}_bin_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

In [609]:
variables.set_index(keys='variable', inplace=True)

integer_vars = create_integer_vars(crops_year_integer, variables)

binary_vars = create_binary_vars(crops_year_binary, variables)

In [610]:
model = pulp.LpProblem("Modelo_1", pulp.LpMaximize)

In [611]:
model += of_net_income(income.loc[0,:], integer_vars, crops_year_integer)

In [612]:
#of_ua_total(integer_vars, crops_year_integer)

In [613]:
#of_crop_diversity(binary_vars, crops_year_binary)

In [614]:
const_idx = add_constraints(model, integer_vars, constraints, crops_year_integer, len(crops), YEARS, const_id = 0)

In [615]:
const_idx = add_binary_correlation(model, binary_vars, integer_vars, 113, const_id = const_idx)

In [616]:
solution = model.solve(pulp.GLPK())

print("\nStatus = {} \nValue = {}".format(pulp.LpStatus[solution], pulp.value(model.objective)))

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/0302455f0d7d48f3a780bd145aadf60a-pulp.lp -o /tmp/0302455f0d7d48f3a780bd145aadf60a-pulp.sol
Reading problem data from '/tmp/0302455f0d7d48f3a780bd145aadf60a-pulp.lp'...
86 rows, 48 columns, 424 non-zeros
48 integer variables, 24 of which are binary
290 lines were read
GLPK Integer Optimizer 5.0
86 rows, 48 columns, 424 non-zeros
48 integer variables, 24 of which are binary
Preprocessing...
24 constraint coefficient(s) were reduced
62 rows, 48 columns, 236 non-zeros
48 integer variables, 24 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.457e-01  max|aij| =  2.244e+00  ratio =  5.033e+00
EQ: min|aij| =  2.114e-01  max|aij| =  1.000e+00  ratio =  4.731e+00
2N: min|aij| =  1.104e-01  max|aij| =  1.562e+00  ratio =  1.416e+01
Constructing initial basis...
Size of triangular part is 62
Solving LP relaxation...
GLPK Simplex Optimizer 5.

12559922.4 | 6332504

In [617]:
model

Modelo_1:
MAXIMIZE
1080.0*y1_int_Autumn_Potatoes + 1013.6*y1_int_Beetroot + 2206.4*y1_int_Broad_Bean + 907.2*y1_int_Cloves + 909.6*y1_int_Cucumbers + 412.0*y1_int_Green_Onion + 1400.0*y1_int_Maize + 2488.0*y1_int_Okra + 1048.8*y1_int_Sunflower + 1252.0*y1_int_Tomatoes + 1092.0*y1_int_Watermelon + 84.0*y1_int_Wheat + 1080.0*y2_int_Autumn_Potatoes + 1013.6*y2_int_Beetroot + 2206.4*y2_int_Broad_Bean + 907.2*y2_int_Cloves + 909.6*y2_int_Cucumbers + 412.0*y2_int_Green_Onion + 1400.0*y2_int_Maize + 2488.0*y2_int_Okra + 1048.8*y2_int_Sunflower + 1252.0*y2_int_Tomatoes + 1092.0*y2_int_Watermelon + 84.0*y2_int_Wheat + 0.0
SUBJECT TO
C_0_C_1: y1_int_Autumn_Potatoes + y1_int_Beetroot + y1_int_Broad_Bean
 + y1_int_Cloves + y1_int_Green_Onion + y1_int_Wheat <= 2395

C_1_C_2: y1_int_Cucumbers + y1_int_Maize + y1_int_Okra + y1_int_Sunflower
 + y1_int_Tomatoes + y1_int_Watermelon <= 2260

C_2_C_3: 100 y1_int_Autumn_Potatoes + 50 y1_int_Beetroot
 + 50 y1_int_Broad_Bean + 50 y1_int_Cloves + 100 y1_int_C